<a href="https://colab.research.google.com/github/sulphatet/GetARoom_MLHackathlon/blob/main/roomRegress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data =pd.read_csv("drive/My Drive/Data/train_DE1.csv")
test_data =pd.read_csv("drive/My Drive/Data/test_DE1.csv")

In [ ]:
data.head()

,Property_ID,Property_Type,Property_Area,Number_of_Windows,Number_of_Doors,Furnishing,Frequency_of_Powercuts,Power_Backup,Water_Supply,Traffic_Density_Score,Crime_Rate,Dust_and_Noise,Air_Quality_Index,Neighborhood_Review,Habitability_score
0,0x21e3,Apartment,106,NaN,1,Semi_Furnished,0.0,No,Once in a day - Morning,5.89,Slightly below average,Medium,90.0,3.86,71.98
1,0x68d4,Apartment,733,2.0,2,Unfurnished,1.0,No,Once in a day - Evening,4.37,Well below average,Medium,96.0,3.55,71.20
2,0x7d81,Apartment,737,4.0,2,Fully Furnished,0.0,No,Once in a day - Morning,7.45,Slightly below average,Medium,121.0,3.81,71.39
3,0x7a57,Apartment,900,3.0,2,Unfurnished,2.0,Yes,Once in a day - Morning,6.16,Well above average,Medium,100.0,1.34,31.46
4,0x9409,Bungalow,2238,14.0,6,Fully Furnished,0.0,No,All time,5.46,Well below average,Medium,116.0,4.77,93.70


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39499 entries, 0 to 39498
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Property_ID             39499 non-null  object 
 1   Property_Type           39499 non-null  object 
 2   Property_Area           39499 non-null  int64  
 3   Number_of_Windows       37845 non-null  float64
 4   Number_of_Doors         39499 non-null  int64  
 5   Furnishing              38457 non-null  object 
 6   Frequency_of_Powercuts  38116 non-null  float64
 7   Power_Backup            39499 non-null  object 
 8   Water_Supply            39499 non-null  object 
 9   Traffic_Density_Score   39499 non-null  float64
 10  Crime_Rate              38712 non-null  object 
 11  Dust_and_Noise          38280 non-null  object 
 12  Air_Quality_Index       39499 non-null  float64
 13  Neighborhood_Review     39499 non-null  float64
 14  Habitability_score      39499 non-null

In [ ]:
data['Property_Type'].unique()

array(['Apartment', 'Bungalow', 'Single-family home', 'Duplex',
       '#R%$G&867', 'Container Home'], dtype=object)

In [ ]:
data['Property_Type'].replace(['Apartment', 'Bungalow', 'Single-family home', 'Duplex','#R%$G&867', 'Container Home'],[1,2,3,4,0,5])

0        1
1        1
2        1
3        1
4        2
        ..
39494    3
39495    1
39496    2
39497    3
39498    3
Name: Property_Type, Length: 39499, dtype: int64

In [ ]:
data['Furnishing'].unique()

array(['Semi_Furnished', 'Unfurnished', 'Fully Furnished', nan],
      dtype=object)

In [ ]:
data['Furnishing'].replace(['Semi_Furnished', 'Unfurnished', 'Fully Furnished', 'nan'],[1,2,3,0])

0        1.0
1        2.0
2        3.0
3        2.0
4        3.0
        ... 
39494    NaN
39495    3.0
39496    2.0
39497    1.0
39498    3.0
Name: Furnishing, Length: 39499, dtype: float64

In [ ]:
#data['Water_Supply'].unique()
#data['Crime_Rate'].unique()
#data['Dust_and_Noise'].unique()
data['Power_Backup'].unique()

array(['No', 'Yes', 'NOT MENTIONED'], dtype=object)

In [ ]:
data['Water_Supply'].replace(['Once in a day - Morning', 'Once in a day - Evening', 'All time','NOT MENTIONED', 'Once in two days'],[1,2,3,0,4])
data['Crime_Rate'].replace(['Slightly below average', 'Well below average','Well above average', 'nan', 'Slightly above average'],[1,2,3,0,4])
data['Dust_and_Noise'].replace(['Medium', 'nan', 'High', 'Low'],[2,0,3,1])

0        2.0
1        2.0
2        2.0
3        2.0
4        2.0
        ... 
39494    2.0
39495    2.0
39496    2.0
39497    NaN
39498    3.0
Name: Dust_and_Noise, Length: 39499, dtype: float64

In [ ]:
data.head()

,Property_ID,Property_Type,Property_Area,Number_of_Windows,Number_of_Doors,Furnishing,Frequency_of_Powercuts,Power_Backup,Water_Supply,Traffic_Density_Score,Crime_Rate,Dust_and_Noise,Air_Quality_Index,Neighborhood_Review,Habitability_score
0,0x21e3,Apartment,106,NaN,1,Semi_Furnished,0.0,No,Once in a day - Morning,5.89,Slightly below average,Medium,90.0,3.86,71.98
1,0x68d4,Apartment,733,2.0,2,Unfurnished,1.0,No,Once in a day - Evening,4.37,Well below average,Medium,96.0,3.55,71.20
2,0x7d81,Apartment,737,4.0,2,Fully Furnished,0.0,No,Once in a day - Morning,7.45,Slightly below average,Medium,121.0,3.81,71.39
3,0x7a57,Apartment,900,3.0,2,Unfurnished,2.0,Yes,Once in a day - Morning,6.16,Well above average,Medium,100.0,1.34,31.46
4,0x9409,Bungalow,2238,14.0,6,Fully Furnished,0.0,No,All time,5.46,Well below average,Medium,116.0,4.77,93.70


In [ ]:
for val, item in data.iteritems():
  print(val)
  #print(item)

Property_ID
Property_Type
Property_Area
Number_of_Windows
Number_of_Doors
Furnishing
Frequency_of_Powercuts
Power_Backup
Water_Supply
Traffic_Density_Score
Crime_Rate
Dust_and_Noise
Air_Quality_Index
Neighborhood_Review
Habitability_score


In [ ]:
test_data.head()

,Property_ID,Property_Type,Property_Area,Number_of_Windows,Number_of_Doors,Furnishing,Frequency_of_Powercuts,Power_Backup,Water_Supply,Traffic_Density_Score,Crime_Rate,Dust_and_Noise,Air_Quality_Index,Neighborhood_Review
0,0x6e93,Apartment,293,3.0,1,Unfurnished,0.0,No,Once in a day - Morning,7.28,Well above average,Medium,152.0,2.52
1,0x8787,Apartment,586,4.0,1,Semi_Furnished,0.0,No,Once in a day - Evening,7.63,Well below average,Medium,92.0,4.16
2,0x6c17,Container Home,305,1.0,2,Semi_Furnished,1.0,No,All time,5.39,Slightly above average,Medium,90.0,2.92
3,0x9dbd,Apartment,258,2.0,1,Semi_Furnished,1.0,No,All time,7.53,Slightly below average,Medium,158.0,3.45
4,0xbfde,Bungalow,3031,12.0,4,Fully Furnished,0.0,No,All time,8.79,Well above average,High,186.0,2.72


In [ ]:
from sklearn.model_selection import train_test_split
data_train,data_test = train_test_split(data)

In [ ]:
len(data_train)

29624

In [ ]:
len(data_test)

9875

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import RobustScaler

data_train['Furnishing'] = data_train['Furnishing'].replace(['Semi_Furnished', 'Unfurnished', 'Fully Furnished', np.NaN],[1,2,3,0])
data_train['Property_Type'] = data['Property_Type'].replace(['Apartment', 'Bungalow', 'Single-family home', 'Duplex','#R%$G&867', 'Container Home'],[1,2,3,4,0,5])
data_train['Water_Supply'] = data_train['Water_Supply'].replace(['Once in a day - Morning', 'Once in a day - Evening', 'All time','NOT MENTIONED', 'Once in two days'],[1,2,3,0,4])
data_train['Crime_Rate'] = data_train['Crime_Rate'].replace(['Slightly below average', 'Well below average','Well above average', np.NaN, 'Slightly above average'],[1,2,3,0,4])
data_train['Dust_and_Noise'] = data['Dust_and_Noise'].replace(['Medium', np.NaN, 'High', 'Low'],[2,0,3,1])
data_train['Power_Backup'] = data['Power_Backup'].replace(['No', 'Yes','NOT MENTIONED'],[0,1,-1])
#data_train['Frequency_of_Powercuts'] = data['Frequency_of_Powercuts'].replace([np.NaN]],[0.5])

data_train = data_train.replace(np.NaN,0)

X_train = data_train.iloc[:,1:-1]
Y_train = data_train['Habitability_score']
#scaler = preprocessing.StandardScaler().fit(X_train)
transformer = RobustScaler().fit(X_train)

In [ ]:
#X_scaled = scaler.transform(X_train)
#X_scaled = transformer.transform(X_train)


In [ ]:
#X_train = X_scaled

In [ ]:
data_test['Furnishing'] = data['Furnishing'].replace(['Semi_Furnished', 'Unfurnished', 'Fully Furnished', np.NaN],[1,2,3,0])
data_test['Property_Type'] = data['Property_Type'].replace(['Apartment', 'Bungalow', 'Single-family home', 'Duplex','#R%$G&867', 'Container Home'],[1,2,3,4,0,5])
data_test['Water_Supply'] = data['Water_Supply'].replace(['Once in a day - Morning', 'Once in a day - Evening', 'All time','NOT MENTIONED', 'Once in two days'],[1,2,3,0,4])
data_test['Crime_Rate'] = data['Crime_Rate'].replace(['Slightly below average', 'Well below average','Well above average', np.NaN, 'Slightly above average'],[1,2,3,0,4])
data_test['Dust_and_Noise'] = data['Dust_and_Noise'].replace(['Medium', np.NaN, 'High', 'Low'],[2,0,3,1])
data_test['Power_Backup'] = data['Power_Backup'].replace(['No', 'Yes','NOT MENTIONED'],[0,1,-1])
#data_train['Frequency_of_Powercuts'] = data['Frequency_of_Powercuts'].replace([np.NaN]],[0.5])

data_test = data_test.replace(np.NaN,0)
X_test = data_test.iloc[:,1:-1]
Y_test = data_test['Habitability_score'] 
#scaler = preprocessing.StandardScaler().fit(X_test)
#X_scaled = scaler.transform(X_test)
#X_test = X_scaled
#transformer = RobustScaler().fit(X_test)
#X_scaled = transformer.transform(X_test)
#X_test = X_scaled

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
model = LinearRegression()
clf1 = model.fit(X_train,Y_train)
model = RandomForestRegressor()
clf2 = model.fit(X_train,Y_train)


In [ ]:
model = RandomForestRegressor(n_estimators=150)
clf3 = model.fit(X_train,Y_train)

In [ ]:
Y_pred = clf1.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score
score = r2_score(Y_test , Y_pred)

In [ ]:
print(score)

0.38915959024972646


In [ ]:
Y_pred = clf2.predict(X_test)
print(r2_score(Y_test,Y_pred))

0.8248536109766242


In [ ]:
Y_pred = clf3.predict(X_test)
print(r2_score(Y_test,Y_pred))

0.8244706038786362


In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()
#clf4 = model.fit(X_train,Y_train)

In [ ]:
#Y_pred = clf4.predict(X_test)
#print(r2_score(Y_test,Y_pred))

0.6463765668895081


In [ ]:
#from sklearn.ensemble import HistGradientBoostingRegressor
#model = HistGradientBoostingRegressor()
#clf5 = model.fit(X_train,Y_train)

In [ ]:
#Y_pred = clf5.predict(X_test)
#print(r2_score(Y_test,Y_pred))

0.8075515747301446


In [ ]:
from sklearn.ensemble import StackingRegressor
#estimators = [('lr2',clf2 ),('lr5', clf5)]
#reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,random_state=42))

In [ ]:
#clf6 = reg.fit(X_train, Y_train)

In [ ]:
#Y_pred = clf6.predict(X_test)
#print(r2_score(Y_test,Y_pred))

In [ ]:
test_data.head()

In [ ]:
test_data['Furnishing'] = test_data['Furnishing'].replace(['Semi_Furnished', 'Unfurnished', 'Fully Furnished', np.NaN],[1,2,3,0])
test_data['Property_Type'] = test_data['Property_Type'].replace(['Apartment', 'Bungalow', 'Single-family home', 'Duplex','#R%$G&867', 'Container Home'],[1,2,3,4,0,5])
test_data['Water_Supply'] = test_data['Water_Supply'].replace(['Once in a day - Morning', 'Once in a day - Evening', 'All time','NOT MENTIONED', 'Once in two days'],[1,2,3,0,4])
test_data['Crime_Rate'] = test_data['Crime_Rate'].replace(['Slightly below average', 'Well below average','Well above average', np.NaN, 'Slightly above average'],[1,2,3,0,4])
test_data['Dust_and_Noise'] = test_data['Dust_and_Noise'].replace(['Medium', np.NaN, 'High', 'Low'],[2,0,3,1])
test_data['Power_Backup'] = test_data['Power_Backup'].replace(['No', 'Yes','NOT MENTIONED'],[0,1,-1])
#data_train['Frequency_of_Powercuts'] = data['Frequency_of_Powercuts'].replace([np.NaN]],[0.5])

test_data = test_data.replace(np.NaN,0)
X_test = test_data.iloc[:,1:]
firstCol = data_test['Property_ID']

In [ ]:
test_data.head()

,Property_ID,Property_Type,Property_Area,Number_of_Windows,Number_of_Doors,Furnishing,Frequency_of_Powercuts,Power_Backup,Water_Supply,Traffic_Density_Score,Crime_Rate,Dust_and_Noise,Air_Quality_Index,Neighborhood_Review
0,0x6e93,1,293,3.0,1,2,0.0,0,1,7.28,3,2,152.0,2.52
1,0x8787,1,586,4.0,1,1,0.0,0,2,7.63,2,2,92.0,4.16
2,0x6c17,5,305,1.0,2,1,1.0,0,3,5.39,4,2,90.0,2.92
3,0x9dbd,1,258,2.0,1,1,1.0,0,3,7.53,1,2,158.0,3.45
4,0xbfde,2,3031,12.0,4,3,0.0,0,3,8.79,3,3,186.0,2.72


In [ ]:
X_test

,Property_Type,Property_Area,Number_of_Windows,Number_of_Doors,Furnishing,Frequency_of_Powercuts,Power_Backup,Water_Supply,Traffic_Density_Score,Crime_Rate,Dust_and_Noise,Air_Quality_Index,Neighborhood_Review
0,1,293,3.0,1,2,0.0,0,1,7.28,3,2,152.0,2.52
1,1,586,4.0,1,1,0.0,0,2,7.63,2,2,92.0,4.16
2,5,305,1.0,2,1,1.0,0,3,5.39,4,2,90.0,2.92
3,1,258,2.0,1,1,1.0,0,3,7.53,1,2,158.0,3.45
4,2,3031,12.0,4,3,0.0,0,3,8.79,3,3,186.0,2.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,1,961,0.0,3,2,0.0,0,2,5.44,1,2,94.0,3.55
10496,5,282,1.0,2,1,3.0,1,3,6.72,2,2,106.0,2.90
10497,4,27445,0.0,2,3,0.0,0,2,9.28,2,3,149.0,4.02
10498,3,1451,2.0,3,2,1.0,1,2,6.25,2,2,131.0,3.49


In [ ]:
Y_pred = clf3.predict(X_test)

In [ ]:
Y_pred

array([31.54686667, 80.40773333, 61.92853333, ..., 75.1928    ,
       79.53113333, 81.52966667])

In [ ]:
firstCol

In [ ]:
firstCol.to_csv(index=False)

In [ ]:
!pwd

In [ ]:
from pathlib import Path  

filepath = Path('/content/out.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

firstCol.to_csv(filepath) 

In [ ]:
series = pd.Series(Y_pred)

filepath = Path('/content/in.csv')  

filepath.parent.mkdir(parents=True, exist_ok=True)  

series.to_csv(filepath) 